In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()
url = "https://www.nseindia.com/market-data/live-equity-market?symbol=NIFTY%2050"
driver.get(url)
#wait until the table gets rendered
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.ID, "equityStockTable")))
page_contents=driver.page_source

In [ ]:
if "TATACONSUM" not in page_contents:
    raise Exception("Page after submit not loaded")

In [ ]:
soup = BeautifulSoup(page_contents, "html.parser")
nse_data = {
    "company_names":[],
    "opening_price":[],
    "high_price":[],
    "low_price":[],
}
table_container = soup.find("table", attrs={"id": "equityStockTable"}).find("tbody")
rows = list(table_container.children)

#for loop to populate nse_data
for i in range(len(rows)):
    if i==0: continue
    valid_tag_count=0
    for child in rows[i].children:
        if(valid_tag_count==4):
            break
        if(child.name is not None):
            if(valid_tag_count==0):
                nse_data["company_names"].append(child.text.strip())
            elif(valid_tag_count==1):
                nse_data["opening_price"].append(child.text.strip())
            elif(valid_tag_count==2):
                nse_data["high_price"].append(child.text.strip())
            elif(valid_tag_count==3):
                nse_data["low_price"].append(child.text.strip())
            valid_tag_count+=1

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(nse_data)
print(df)

In [ ]:
#close driver and export bse_data
driver.quit()
%store nse_data

In [ ]:
import time
import pandas as pd
driver = webdriver.Chrome();
one_year_data_nse = {
    "date": [],
    "open": [],
    "high": [],
    "low": [],
    "close": []
}
one_year_url="https://www.nseindia.com/reports-indices-historical-index-data"
driver.get(one_year_url)

select_element=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, "hpReportIndexTypeSearchInput")))
time.sleep(2)
select_element.click()

options=select_element.find_elements(By.TAG_NAME, "option")
for option in options:
    if option.text.strip()=="NIFTY 50":
        option.click()
        break

time_filter_options=driver.find_elements(By.CSS_SELECTOR, "#hrFilterContainer .col-md-12 ul li")
for time_option in time_filter_options:
    if time_option.text.strip()=="1Y":
        time_option.click()
        break

table=WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#historic-repot-table-reference tbody")))
time.sleep(5)
rows = table.find_elements(By.CSS_SELECTOR, "tbody tr")
print("row count:", len(rows))
time.sleep(3)
for index in range(len(rows)):
    row = table.find_elements(By.CSS_SELECTOR, "tbody tr")
    cols = row[index].find_elements(By.TAG_NAME, "td")
    for i in range(5):
        if(i==0):
            one_year_data_nse["date"].append(cols[i].text.strip())
        if(i==1):
            one_year_data_nse["open"].append(cols[i].text.strip())
        if(i==2):
            one_year_data_nse["high"].append(cols[i].text.strip())
        if(i==3):
            one_year_data_nse["low"].append(cols[i].text.strip())
        if(i==4):
            one_year_data_nse["close"].append(cols[i].text.strip())
df=pd.DataFrame.from_dict(one_year_data_nse)
df.head()

In [ ]:
driver.quit()
%store one_year_data_nse